In [2]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import joblib
import time
import json

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, TimeDistributed, Flatten, RepeatVector, InputLayer, Reshape
import visualkeras

In [95]:
class CNNLSTMModel:
    def __init__(self, processor):
        self.processor = processor
        self.model = None
        self.build_model()

    def build_model(self):
        self.model = Sequential()

        t_i = self.processor.t_i
        M = self.processor.M
        N_t = self.processor.N_t
        N_r = self.processor.N_r
        t_o = self.processor.t_o

        self.model.add(InputLayer(input_shape=(t_i, M, 2*N_t*N_r)))
        self.model.add(Conv2D(64, kernel_size=(3, 1), padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 1)))
        self.model.add(Conv2D(64, kernel_size=(3, 1), padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(RepeatVector(t_o))
        self.model.add(LSTM(16, activation='tanh', return_sequences=True))
        self.model.add(LSTM(16, activation='tanh', return_sequences=True))
        self.model.add(TimeDistributed(Dense(4)))
        self.model.add(Reshape((t_o, M, 2*N_t*N_r)))
        self.model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
        self.model.summary()

    def train_model(self, X_train, y_train, batch_size = 64, epochs=20):
        return self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    
    def predict_model(self, X_test):
        return self.model.predict(X_test)
    
    def evaluate_model(self, x_test, y_test):
        test_loss, test_acc = self.model.evaluate(x_test, y_test)
        return test_loss, test_acc

In [6]:
class DatasetProcessor:
    def __init__(self, file_path, t_i, t_o, N_t, N_r):
        self.file_path = file_path
        self.t_i = t_i  # Number of input time steps
        self.t_o = t_o  # Number of output time steps
        self.N_t = N_t  # Number of transmit antennas
        self.N_r = N_r  # Number of receive antennas
        self.M = None
        self.K = None
        self.scaler_X = MinMaxScaler()
        self.scaler_y = MinMaxScaler()

    def load_data(self):
        # Load the .mat dataset
        data = scipy.io.loadmat(self.file_path)
        Y = data["Y"]
        G = data["G"]
        SNR = data["SNR_dB"].flatten()
        Q = data["Q"].flatten()
        if Y.ndim == 4:
            Y = np.transpose(Y, (2, 1, 0, 3))
            self.SNR = Y.shape[3]
        else:
            Y = np.transpose(Y, (2, 1, 0))
        if G.ndim == 4:
            G = np.transpose(G, (2, 1, 0, 3))
        else:
            G = np.transpose(G, (2, 1, 0))
        return Y, G, SNR, Q
    
    def preprocess_data(self, Y, G):
        self.K = Y.shape[0]  # Number of timesteps
        self.M = Y.shape[1]  # Number of users
        Sample = self.K - self.t_i - self.t_o + 1
        print(Sample)

        #amplitude_G = G[:,:,0]

        # Create sequences for input and output
        X = np.array([Y[i:i+self.t_i] for i in range(Sample)])
        y = np.array([G[i + self.t_i:i + self.t_i + self.t_o] for i in range(Sample)])
        return X, y
    
    def sequential_split_data(self, X, y, test_size=0.2):
        # Calculate split indices
        num_samples = X.shape[0]
        test_split_index = int(num_samples * (1 - test_size))

        # Perform sequential splits
        X_train, y_train = X[:test_split_index], y[:test_split_index]
        X_test, y_test = X[test_split_index:], y[test_split_index:]

        return X_train, y_train, X_test, y_test

    def normalize_data(self, X_train, y_train, X_test, y_test):
        # # Normalize data
        X_train_scaled = self.scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_test_scaled = self.scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

        y_train_scaled = self.scaler_y.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
        y_test_scaled = self.scaler_y.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

        # Save the scalers for future use
        joblib.dump(self.scaler_X, 'scaler_X.pkl')
        joblib.dump(self.scaler_y, 'scaler_y.pkl')

        # return X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled
        # X_scaled = self.scaler_X.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        # y_scaled = self.scaler_y.fit_transform(y.reshape(-1, y.shape[-1])).reshape(y.shape)

        return X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled



In [41]:
def grid_search_Nfilters(processor, X_train, y_train, X_test, y_test, Nfilter1_values, Nfilter2_values):
        results = []
        
        for Nfilter1 in Nfilter1_values:
            for Nfilter2 in Nfilter2_values:
                
                # Initialize and build the model
                model = CNNLSTMModel(processor, Nfilter1, Nfilter2)
                
                # Train the model
                model.train_model(X_train, y_train)
                
                # Evaluate the model
                test_loss, test_acc = model.evaluate_model(X_test, y_test)
                
                # Store the results
                results.append({
                    'Nfilter1': Nfilter1,
                    'Nfilter2': Nfilter2,
                    'test_loss': test_loss,
                    'test_acc': test_acc
                })
                
                print(f"Nfilter1={Nfilter1}, Nfilter2={Nfilter2}: Test Loss = {test_loss}, Test Accuracy = {test_acc}")
        
        return results

# Nfilter1=64, Nfilter2=64: Test Loss = 0.012378140352666378, Test Accuracy = 0.7205955982208252
# Nfilter1=4, Nfilter2=4: Test Loss = 0.013206719420850277, Test Accuracy = 0.7143393158912659
# Nfilter1=4, Nfilter2=8: Test Loss = 0.01393490843474865, Test Accuracy = 0.7012012004852295
# Nfilter1=4, Nfilter2=16: Test Loss = 0.013972993940114975, Test Accuracy = 0.6999499201774597
# Nfilter1=4, Nfilter2=64: Test Loss = 0.01888369955122471, Test Accuracy = 0.6763013005256653
# Nfilter1=8, Nfilter2=4: Test Loss = 0.012817522510886192, Test Accuracy = 0.7173423171043396
# Nfilter1=8, Nfilter2=8: Test Loss = 0.013387846760451794, Test Accuracy = 0.7117117047309875
# Nfilter1=8, Nfilter2=16: Test Loss = 0.014078735373914242, Test Accuracy = 0.6958208084106445
# Nfilter1=8, Nfilter2=64: Test Loss = 0.016480514779686928, Test Accuracy = 0.6831831932067871
# Nfilter1=16, Nfilter2=4: Test Loss = 0.012622952461242676, Test Accuracy = 0.7204704880714417
# Nfilter1=16, Nfilter2=8: Test Loss = 0.013172678649425507, Test Accuracy = 0.7109609842300415
# Nfilter1=16, Nfilter2=16: Test Loss = 0.014708010479807854, Test Accuracy = 0.6965715885162354
# Nfilter1=16, Nfilter2=64: Test Loss = 0.018291575834155083, Test Accuracy = 0.6771771907806396
# Nfilter1=64, Nfilter2=4: Test Loss = 0.012815110385417938, Test Accuracy = 0.7150900959968567
# Nfilter1=64, Nfilter2=8: Test Loss = 0.015085449442267418, Test Accuracy = 0.7022022008895874
# Nfilter1=64, Nfilter2=16: Test Loss = 0.01513038668781519, Test Accuracy = 0.6931931972503662
# Nfilter1=64, Nfilter2=64: Test Loss = 0.012378140352666378, Test Accuracy = 0.7205955982208252


In [30]:
def batch_size_grid_search(processor, X_train, y_train, X_val_scaled, y_val_scaled, X_test, y_test, batch_size_values):
    results = []

    for batch_size in batch_size_values:
        # Initialize and build the model
        model = CNNLSTMModel(processor)
        
        # Train the model
        model.train_model(X_train, y_train, X_val_scaled, y_val_scaled, batch_size)
        
        # Evaluate the model
        test_loss, test_acc = model.evaluate_model(X_test, y_test)
        
        # Store the results
        results.append({
            'batch_size': batch_size,
            'test_loss': test_loss,
            'test_acc': test_acc
        })
        
        print(f"batch_size={batch_size}: Test Loss = {test_loss}, Test Accuracy = {test_acc}")
        
    return results

# batch_size=4: Test Loss = 3.1704727007308975e-05, Test Accuracy = 0.5387887954711914
# batch_size=16: Test Loss = 2.9865463147871196e-05, Test Accuracy = 0.5387887954711914
# batch_size=32: Test Loss = 3.1243307603290305e-05, Test Accuracy = 0.5415415167808533
# batch_size=64: Test Loss = 1.3201074580138084e-05, Test Accuracy = 0.532282292842865

In [156]:
processor = DatasetProcessor("dataset_3.mat", 20, 1, 1, 1)
Y, G, SNR, Q = processor.load_data()

if Y.ndim == 4:
    # Initialize arrays to hold all results
    y_pred_all = None
    y_true_all = None
    X_all = None
    y_pred_test = None
    y_pred_train_save = None
    y_true_test = None
    y_true_train = None
    X_test_save = None
    X_train_save = None
    num_snr = len(SNR)
    num_mod = len(Q)

    inference_times_per_snr = {}  # Dictionary to store inference times for each SNR
    history_per_snr = {}  # Dictionary to store training history for each SNR

    for snr_idx, snr in enumerate(SNR):
        for mod_idx, mod in enumerate(Q):
            print('snr:', snr_idx, snr)
            new_Y = Y[:, :, :, snr_idx].reshape(Y.shape[0], Y.shape[1], Y.shape[2])
            new_G = G[:, :, :, snr_idx].reshape(G.shape[0], G.shape[1], G.shape[2])

            ## Preprocessing
            X, y = processor.preprocess_data(new_Y, new_G)

            # Normalize before then split
            X_train, y_train, X_test, y_test = processor.sequential_split_data(X, y)
            X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled = processor.normalize_data(X_train, y_train, X_test, y_test)

            print(X_train_scaled.shape, y_train_scaled.shape, X_test_scaled.shape, y_test_scaled.shape)
            ## Start model 
            model = CNNLSTMModel(processor)
            history = model.train_model(X_train_scaled, y_train_scaled)

            history_per_snr[f"SNR_{snr}"] = history.history

            # Measure total inference time for all predictions
            start_time = time.perf_counter()
            y_pred = model.predict_model(X_test_scaled)
            total_inference_time = time.perf_counter() - start_time  # Total prediction time for this SNR

            # Calculate prediction time per time step
            num_time_steps = X_test_scaled.shape[0]
            predict_time_per_time_step = total_inference_time / num_time_steps

            # Store the inference time for this SNR
            inference_times_per_snr[f"SNR_{snr}"] = predict_time_per_time_step
            
            y_pred_train = model.predict_model(X_train_scaled)

            y_pred_train_original = processor.scaler_y.inverse_transform(y_pred_train.reshape(-1, y_pred_train.shape[-1])).reshape(y_pred_train.shape)
            y_pred_test_original = processor.scaler_y.inverse_transform(y_pred.reshape(-1, y_pred.shape[-1])).reshape(y_pred.shape)

            y_true_train_original = processor.scaler_y.inverse_transform(y_train_scaled.reshape(-1, y_train_scaled.shape[-1])).reshape(y_train.shape)
            y_true_test_original = processor.scaler_y.inverse_transform(y_test_scaled.reshape(-1, y_test_scaled.shape[-1])).reshape(y_test.shape)
            
            X_train_original = processor.scaler_X.inverse_transform(X_train_scaled.reshape(-1, X_train_scaled.shape[-1])).reshape(X_train.shape)
            X_test_original = processor.scaler_X.inverse_transform(X_test_scaled.reshape(-1, X_test_scaled.shape[-1])).reshape(X_test.shape)

            concatenated_pred_original = np.concatenate((y_pred_train_original, y_pred_test_original))
            concatenated_y_original = np.concatenate((y_true_train_original, y_true_test_original))
            concatenated_X_original = np.concatenate((X_train_original, X_test_original))

            num_samples = concatenated_pred_original.shape[0]
            
            if y_pred_all is None:
                y_pred_all = np.zeros((num_samples, concatenated_pred_original.shape[1], concatenated_pred_original.shape[2], concatenated_pred_original.shape[3], num_snr))
                y_true_all = np.zeros((num_samples, concatenated_y_original.shape[1], concatenated_y_original.shape[2], concatenated_y_original.shape[3], num_snr))
                X_all = np.zeros((num_samples, concatenated_X_original.shape[1], concatenated_X_original.shape[2], concatenated_X_original.shape[3], num_snr))

                y_pred_test = np.zeros((y_pred_test_original.shape[0], y_pred_test_original.shape[1], y_pred_test_original.shape[2], y_pred_test_original.shape[3], num_snr))
                y_pred_train_save = np.zeros((y_pred_train_original.shape[0], y_pred_train_original.shape[1], y_pred_train_original.shape[2], y_pred_train_original.shape[3], num_snr))

                y_true_test = np.zeros((y_true_test_original.shape[0], y_true_test_original.shape[1], y_true_test_original.shape[2], y_true_test_original.shape[3], num_snr))
                y_true_train = np.zeros((y_true_train_original.shape[0], y_true_train_original.shape[1], y_true_train_original.shape[2], y_true_train_original.shape[3], num_snr))

                X_test_save = np.zeros((X_test_original.shape[0], X_test_original.shape[1], X_test_original.shape[2], X_test_original.shape[3], num_snr))
                X_train_save = np.zeros((X_train_original.shape[0], X_train_original.shape[1], X_train_original.shape[2], X_train_original.shape[3], num_snr))

            y_pred_all[:, :, :, :, snr_idx] = concatenated_pred_original
            y_true_all[:, :, :, :, snr_idx] = concatenated_y_original
            X_all[:, :, :, :, snr_idx] = concatenated_X_original
            y_pred_test[:, :, :, :, snr_idx] = y_pred_test_original
            y_pred_train_save[:, :, :, :, snr_idx] = y_pred_train_original
            y_true_test[:, :, :, :, snr_idx] = y_true_test_original
            y_true_train[:, :, :, :, snr_idx] = y_true_train_original
            X_test_save[:, :, :, :, snr_idx] = X_test_original
            X_train_save[:, :, :, :, snr_idx] = X_train_original
    
    # Save inference times to a JSON file
    with open('./results_model_1/inference_times_per_snr_1.json', 'w') as json_file:
        json.dump(inference_times_per_snr, json_file)

    # Save training history to a JSON file
    with open('./results_model_1/history_per_snr_1.json', 'w') as json_file:
        json.dump(history_per_snr, json_file)


    # Save to .mat file
    scipy.io.savemat('./results_model_1/results_2_users_dl_16QAM_1.mat', {
        'y_pred_all': y_pred_all,
        'y_true_all': y_true_all,
        'X_all': X_all,
        'y_pred_test' : y_pred_test,
        'y_pred_train' : y_pred_train_save,
        'y_true_test' : y_true_test,
        'y_true_train' : y_true_train,
        'X_test' : X_test_save,
        'X_train' : X_train_save,
        'SNR_dB': SNR,
        'Q': Q
    })
   
else:
    ## Preprocessing
    X, y = processor.preprocess_data(Y, G)

    # # Split before then Normalize data
    # X_train, y_train, X_test, y_test = processor.sequential_split_data(X, y)
    # X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled = processor.normalize_data(X_train, y_train, X_test, y_test)

    # Normalize before then split
    X_scaled, y_scaled = processor.normalize_data(X,y)
    X_train, y_train, X_test, y_test = processor.sequential_split_data(X_scaled, y_scaled)
    
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    ## Start model 
    model = CNNLSTMModel(processor)
    history = model.train_model(X_train, y_train)

    y_pred = model.predict_model(X_test)
    y_pred_train = model.predict_model(X_train)

    concatenated_pred = np.concatenate((y_pred_train, y_pred))
    concatenated_y = np.concatenate((y_train, y_test))

    # # Inverse transform the predictions and test data for comparison
    # y_test_original = processor.scaler_y.inverse_transform(y_test_scaled.reshape(-5, y_test_scaled.shape[-1])).reshape(y_test_scaled.shape)
    # y_pred_original = processor.scaler_y.inverse_transform(y_pred.reshape(-1, y_pred.shape[-1])).reshape(y_pred.shape)

    # ## Plots
    plotter = GeneratePlots(processor)
    plotter.plot_model_evaluation(y_test, y_pred)
    plotter.plot_training_history(history)
    plotter.plot_model_metrics(y_test, y_pred)

snr: 0 -10
19980
(15984, 20, 2, 2) (15984, 1, 2, 2) (3996, 20, 2, 2) (3996, 1, 2, 2)
Model: "sequential_114"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_228 (Conv2D)         (None, 20, 2, 64)         448       
                                                                 
 max_pooling2d_228 (MaxPool  (None, 10, 2, 64)         0         
 ing2D)                                                          
                                                                 
 conv2d_229 (Conv2D)         (None, 10, 2, 64)         12352     
                                                                 
 max_pooling2d_229 (MaxPool  (None, 5, 1, 64)          0         
 ing2D)                                                          
                                                                 
 flatten_114 (Flatten)       (None, 320)               0         
                                 